In [1]:
!pip install --upgrade pip
!pip install gdown

     |████████████████████████████████| 2.1 MB 620 kB/s 
  Attempting uninstall: pip
    Found existing installation: pip 20.1.1
    Uninstalling pip-20.1.1:
      Successfully uninstalled pip-20.1.1
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - \ | done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14775 sha256=26c5cda8f59396f71b6b60091250c2598e45b7aaf30d2823bdd18542ad63168f
  Stored in directory: /root/.cache/pip/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown


In [2]:
import os
if not (os.path.isfile('data.zip') and os.path.isfile('data.zip')):
    !rm -rf model_discovery
#     !rm -rf data
    import gdown
    # download source code
    gdown.download('https://drive.google.com/uc?id=1exw9vOYcb0fPVseleifK0m1VxbBCsBM_', output=None, quiet=False)
    # download data
#     gdown.download('https://drive.google.com/uc?id=1ozrvNU128WOGWDVhhDKtZENzBMuO-w4i', output=None, quiet=False)

#     !unzip -qq data.zip
    !unzip -qq model_discovery.zip

Downloading...
From: https://drive.google.com/uc?id=1exw9vOYcb0fPVseleifK0m1VxbBCsBM_
To: /kaggle/working/model_discovery.zip
100%|██████████| 11.9k/11.9k [00:00<00:00, 2.99MB/s]


In [3]:
# import deepmatcher as dm
import itertools
import os
import pandas as pd
import model_discovery
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from model_discovery import utils, l2lsh
import re

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet


import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras 
from keras import backend as K
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from keras.layers import LSTM,Dense,Bidirectional,Input
from keras.models import Model
import torch
import transformers

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
# # download fasttext word vector
# gdown.download('https://drive.google.com/uc?id=12tdZ76mb8tT98pz3lAy2gWIPZvmKDTSo', output=None, quiet=False)

In [5]:
datasets = ['../input/imbd-data-1/Movie_1.csv', '../input/financial-news-data-2/Financial_News_2.csv', '../input/tweets-data-4/Tweets_4.csv',
       '../input/tweets-hatred-data-5/Tweets_hatred_5.csv']

In [6]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

def fast_encode(texts, tokenizer, chunk_size=256, maxlen=400):

    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

def build_model(transformer, max_len=400):
    
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=2e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

from tokenizers import BertWordPieceTokenizer
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased' , lower = True)
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
bert_model = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')

In [7]:
def model_serving(src, tar):
    print(src,tar)
    src=pd.read_csv(src).sample(2000, replace=True)
    tar=pd.read_csv(tar).sample(2000, replace=True)
    
    src['text']=src['text'].apply(denoise_text)
    tar['text']=tar['text'].apply(denoise_text)
        
    #---------------------------------------------------------------------------
    
    data1 = utils.flatten_df(src['text'])
    word1 = utils.word_tokenize(data1)
    data2 = utils.flatten_df(tar['text'])
    word2 = utils.word_tokenize(data2)
    
    jsd=utils.jsd_for_word(word1, word2)
    ada=utils.adaptivity_word(src['text'], tar['text'], word1, word2, threshold=0.6)
    l2d=utils.l2d_btw_domains(word1, word2)
    
    #----------------------------------------------------------------------------
    
    x_train_src,x_test_src,y_train_src,y_test_src = train_test_split(src.text,src.label,random_state = 0 , stratify = src.label)
    x_train_tar,x_test_tar,y_train_tar,y_test_tar = train_test_split(tar.text,tar.label,random_state = 0 , stratify = tar.label)

    x_train_src = fast_encode(x_train_src.values, fast_tokenizer, maxlen=400)
    x_test_src = fast_encode(x_test_src.values, fast_tokenizer, maxlen=400)
    
    x_train_tar = fast_encode(x_train_tar.values, fast_tokenizer, maxlen=400)
    x_test_tar = fast_encode(x_test_tar.values, fast_tokenizer, maxlen=400)
    
    model_src = build_model(bert_model, max_len=400)
    model_src.fit(x_train_src,y_train_src,batch_size = 32 ,validation_data=(x_test_src,y_test_src),epochs = 1)
    src_acc = model_src.evaluate(x_test_src,y_test_src, verbose=1)[1]*100
    print(src_acc)

    ser_acc = model_src.evaluate(x_train_tar,y_train_tar, verbose=1)[1]*100
    print(ser_acc)
    
    model_tar = build_model(bert_model, max_len=400)
    model_tar.fit(x_train_tar,y_train_tar,batch_size = 32, epochs = 1)
    ft_acc = model_src.evaluate(x_test_tar,y_test_tar, verbose=1)[1]*100
    print(ft_acc)

    return [jsd,ada,l2d,src_acc, ser_acc, ft_acc]


In [8]:
#src_acc, ser_acc, ft_acc
comb = list(itertools.permutations(datasets, 2))
src_list = []
tar_list = []
src_acc_list = []
ser_acc_list = []
ft_acc_list = []
jsd_list = []
ada_list = []
l2d_list = []
for src, tar in tqdm(comb, leave=False):
#     src_acc, ser_acc, ft_acc = model_serving(src, tar)
    jsd, ada, l2d, src_acc, ser_acc, ft_acc = model_serving(src, tar)
    jsd_list.append(jsd)
    ada_list.append(ada)
    src=re.search('input/(.*).csv', src).group(1).split('/')[1]
    tar=re.search('input/(.*).csv', tar).group(1).split('/')[1]
    src_list.append(src)
    tar_list.append(tar)
    l2d_list.append(l2d)
    src_acc_list.append(src_acc)
    ser_acc_list.append(ser_acc)
    ft_acc_list.append(ft_acc)
#     break

../input/imbd-data-1/Movie_1.csv ../input/financial-news-data-2/Financial_News_2.csv
16/16 [==============================] - 4s 232ms/step - loss: 0.5573 - accuracy: 0.7240
72.39999771118164
47/47 [==============================] - 12s 248ms/step - loss: 0.6041 - accuracy: 0.7480
74.80000257492065
16/16 [==============================] - 4s 232ms/step - loss: 0.5968 - accuracy: 0.6720
67.1999990940094
../input/imbd-data-1/Movie_1.csv ../input/tweets-data-4/Tweets_4.csv
16/16 [==============================] - 4s 233ms/step - loss: 0.5035 - accuracy: 0.7320
73.19999933242798
47/47 [==============================] - 12s 248ms/step - loss: 0.5909 - accuracy: 0.6873
68.7333345413208
16/16 [==============================] - 4s 233ms/step - loss: 0.6296 - accuracy: 0.6640
66.39999747276306
../input/imbd-data-1/Movie_1.csv ../input/tweets-hatred-data-5/Tweets_hatred_5.csv
16/16 [==============================] - 4s 234ms/step - loss: 0.4575 - accuracy: 0.7900
79.00000214576721
47/47 [=======

/kaggle/working/model_discovery/utils.py:356: RuntimeWarning: invalid value encountered in true_divide
  prob1 = prob1 / np.sum(prob1)


16/16 [==============================] - 4s 233ms/step - loss: 0.1701 - accuracy: 0.9400
93.99999976158142
47/47 [==============================] - 12s 248ms/step - loss: 0.3725 - accuracy: 0.8407
84.0666651725769
16/16 [==============================] - 4s 235ms/step - loss: 0.3560 - accuracy: 0.8500
85.00000238418579
../input/financial-news-data-2/Financial_News_2.csv ../input/tweets-data-4/Tweets_4.csv


/kaggle/working/model_discovery/utils.py:356: RuntimeWarning: invalid value encountered in true_divide
  prob1 = prob1 / np.sum(prob1)


16/16 [==============================] - 4s 235ms/step - loss: 0.1421 - accuracy: 0.9460
94.59999799728394
47/47 [==============================] - 12s 249ms/step - loss: 0.8850 - accuracy: 0.6907
69.0666675567627
16/16 [==============================] - 4s 233ms/step - loss: 0.7107 - accuracy: 0.6920
69.19999718666077
../input/financial-news-data-2/Financial_News_2.csv ../input/tweets-hatred-data-5/Tweets_hatred_5.csv


/kaggle/working/model_discovery/utils.py:356: RuntimeWarning: invalid value encountered in true_divide
  prob1 = prob1 / np.sum(prob1)


16/16 [==============================] - 4s 236ms/step - loss: 0.1326 - accuracy: 0.9580
95.80000042915344
47/47 [==============================] - 12s 249ms/step - loss: 0.6483 - accuracy: 0.7660
76.5999972820282
16/16 [==============================] - 4s 234ms/step - loss: 0.3615 - accuracy: 0.8800
87.99999952316284
../input/tweets-data-4/Tweets_4.csv ../input/imbd-data-1/Movie_1.csv
16/16 [==============================] - 4s 236ms/step - loss: 0.5998 - accuracy: 0.6860
68.59999895095825
47/47 [==============================] - 12s 251ms/step - loss: 0.4094 - accuracy: 0.8360
83.60000252723694
16/16 [==============================] - 4s 233ms/step - loss: 0.3793 - accuracy: 0.8380
83.79999995231628
../input/tweets-data-4/Tweets_4.csv ../input/financial-news-data-2/Financial_News_2.csv
16/16 [==============================] - 4s 234ms/step - loss: 0.5959 - accuracy: 0.6900
68.99999976158142
47/47 [==============================] - 12s 248ms/step - loss: 0.2799 - accuracy: 0.9193
91.

In [9]:
result_df = pd.DataFrame({'Source 1': src_list, 'Source 2': tar_list, 'JSD': jsd_list,'Adaptivity': ada_list, 'L2D':l2d_list,
                         'Source Accuracy': src_acc_list, 'Serving Accuracy': ser_acc_list, 'Finetune Accuracy': ft_acc_list})

result_df.to_csv('Sentiment_serving-train.csv', index=False)

In [10]:
result_df

,Source 1,Source 2,JSD,Adaptivity,L2D,Source Accuracy,Serving Accuracy,Finetune Accuracy
0,Movie_1,Financial_News_2,0.482698,3.857143,0.195954,72.399998,74.800003,67.199999
1,Movie_1,Tweets_4,0.136582,6.285714,0.080646,73.199999,68.733335,66.399997
2,Movie_1,Tweets_hatred_5,0.134282,6.428571,0.085405,79.000002,62.933332,77.999997
3,Financial_News_2,Movie_1,0.487951,3.571429,0.199446,94.000000,84.066665,85.000002
4,Financial_News_2,Tweets_4,0.530040,3.428571,0.155057,94.599998,69.066668,69.199997
5,Financial_News_2,Tweets_hatred_5,0.507130,2.571429,0.158835,95.800000,76.599997,88.000000
6,Tweets_4,Movie_1,0.125747,5.571429,0.078769,68.599999,83.600003,83.800000
7,Tweets_4,Financial_News_2,0.538059,1.857143,0.158241,69.000000,91.933334,95.400000
8,Tweets_4,Tweets_hatred_5,0.084045,7.000000,0.053896,68.400002,61.466664,40.799999
9,Tweets_hatred_5,Movie_1,0.135038,6.428571,0.080814,91.799998,49.866667,49.800000
